In [91]:
import pandas as pd
import numpy as np
from joblib import Parallel, delayed
import multiprocessing
from sklearn.metrics import mean_squared_error
from sklearn import preprocessing
from sklearn.linear_model import LassoCV

In [92]:
# Done ✅
def load_dataset(path):
    dataset = pd.read_csv(path)
    dataset = dataset.sample(frac=1).reset_index(drop=True).to_numpy()
    return dataset

In [93]:
# Done ✅
def normal_dataset(dataset):
    dataset = preprocessing.normalize(dataset)
    return dataset

In [94]:
# Done ✅
def create_train_test_data(dataset):
    index = int(0.8 * len(dataset))
    train = dataset[:index]
    test = dataset[index:]
    features, b1, b2 = train[:,:8], train[:,8], train[:,9]
    features_test, b1_test, b2_test = test[:,:8], test[:,8], test[:,9]
    b1 = b1.reshape((-1,1))
    b2 = b2.reshape((-1,1))
    b1_test = b1_test.reshape((-1,1))
    b2_test = b2_test.reshape((-1,1))
    return features, b1, b2,features_test, b1_test, b2_test


In [95]:
# Done ✅
def calculte_x_t(A,b,ro,z,nu,d):
    t1 = np.linalg.inv(A.T.dot(A) + ro * np.identity(d))
    t2 = A.T.dot(b) + ro*z -  nu
    return t1.dot(t2)

In [96]:
# Done ✅
def calculte_z_t(z, x, nu, ro, landa):
    e_grad = x + nu*1.0/ro
    z_t = np.zeros_like(z)
    l = -(1.0*landa/ro)*(z<0)
    g = (1.0*landa/ro)*(z>0)
    z_t = e_grad - l - g
    return z_t

In [97]:
# Done ✅
def calculate_nu_t(nu_t,ro,X_t,z_t):
    nu_t = nu_t + ro*(X_t - z_t)
    return nu_t

In [98]:
# Done ✅
def calculate_value(A,X,b,landa):
    value = 0.5*np.linalg.norm(A.dot(X) - b, ord='fro')**2 + landa*np.linalg.norm(X, ord=1)
    return value

In [99]:
# Done ✅

def print_status(status,A,A_test,X_t,b,b_test,landa,iteration):
    if status == 0:
        print("First value:",calculate_value(A,X_t,b,landa))
        print("--------------------------------")
    elif status == 1:
        print("--------------------------------")
        print("Iteration " + str(iteration+1))
        print("value:",calculate_value(A,X_t,b,landa))
        print("Rmse of Train Data: "+str(mean_squared_error(A.dot(X_t),b)))
        print("Rmse of Test Data: "+str(mean_squared_error(A_test.dot(X_t),b_test)))
        print("--------------------------------")
    else:
        print("--------------------------------")
        print("--------Final Results--------")
        print("Final value:",calculate_value(A,X_t,b,landa))
        print("Rmse of Train Data: "+str(mean_squared_error(A.dot(X_t),b)))
        print("Rmse of Test Data: "+str(mean_squared_error(A_test.dot(X_t),b_test)))
        print("--------------------------------\n")


In [100]:
# Done ✅

def LassoCV_Regresion(A,b,A_test,b_test):
    model = LassoCV(cv=5, random_state=0).fit(A, b)
    model.score(A, b)
    predict_train = model.predict(A)
    predict_test = model.predict(A_test)
    
    print("--------LassoCV Results--------")
    print("Rmse of Train Data: "+str(mean_squared_error(predict_train,b)))
    print("Rmse of Test Data: = "+str(mean_squared_error(predict_test,b_test)))
    print("--------------------------------")

In [101]:
# Done ✅

def none_Parallel_ADMM(A,b,A_test , b_test,d,iterations,landa):
    n = len(A)
    X_t = np.random.randn(d, 1)
    z_t = np.zeros((d,1))
    ro = 1
    nu_t = np.zeros((d,1))
    print_status(0,A,A_test,X_t,b,b_test,landa,0)
    for iteration in range(iterations):
        X_t = calculte_x_t(A,b,ro,z_t,nu_t,d)
        z_t = calculte_z_t(z_t, X_t, nu_t, ro, landa)
        nu_t = calculate_nu_t(nu_t,ro,X_t,z_t)
        print_status(1,A,A_test,X_t,b,b_test,landa,iteration)
        print()
    print_status(2,A,A_test,X_t,b,b_test,landa,iterations)

In [102]:
# Done ✅

def calculate_pararell(i,A,b,nu_t,ro,z_t,X_t):
    ai = A[i].reshape(-1,1).T
    bi = b[i].reshape(-1,1)
    bi = bi.item()
    nui = nu_t[i].reshape(-1,1)
    term1 = 1/(ai.dot(ai.T) + ro)
    term2 = ai.T*bi + ro*z_t - nui
    term1 = term1.item()
    x = term1*term2
    x = x.T
    X_t[i] = x

In [103]:
# ❌
def update_nu_t_pararell(nu_t,X_t,z_t,ro):
    for i in range(nu_t.shape[0]):
        nu_t[i] = (nu_t[i].reshape(-1,1) + ro*(X_t[i].reshape(-1,1) - z_t)).T
    return nu_t

In [104]:
# ❌
def Parallel_ADMM(A,b,A_test , b_test,d,iterations,landa):
    n = len(A)
    X_t = np.tile(np.random.randn(d, 1).T, (n,1))
    z_t = np.zeros((d,1))
    ro = 1
    nu_t = np.tile(np.zeros((d,1)).T, (n,1))

    X = np.mean(X_t, axis=0).reshape(-1,1)
    print_status(0,A,A_test,X,b,b_test,landa,0)

    for iteration in range(iterations):
        Parallel(n_jobs=multiprocessing.cpu_count() , require='sharedmem')(delayed(calculate_pararell)(i,A,b,nu_t,ro,z_t,X_t) for i in range(n))
        X = np.mean(X_t, axis=0).reshape(-1,1)
        nu = np.mean(nu_t, axis=0).reshape(-1,1)
        z_t = calculte_z_t(z_t, X, nu, ro, landa)
        nu_t = update_nu_t_pararell(nu_t,X_t,z_t,ro)
        print_status(1,A,A_test,X,b,b_test,landa,iteration)
    print_status(2,A,A_test,X,b,b_test,landa,iterations)


In [105]:
dataset = load_dataset("ENB2012_data.csv")
dataset_normal = normal_dataset(dataset)
features, b1, b2,features_test, b1_test, b2_test = create_train_test_data(dataset_normal)
d=8

## none_Parallel_ADMM

In [106]:
none_Parallel_ADMM(features,b1,features_test , b1_test,d,100,0.01)

First value: 169.40233446870513
--------------------------------
--------------------------------
Iteration 1
value: 0.022951287220364602
Rmse of Train Data: 6.793472128419149e-05
Rmse of Test Data: 8.544753814443038e-05
--------------------------------

--------------------------------
Iteration 2
value: 0.021138450118061158
Rmse of Train Data: 6.01666016700233e-05
Rmse of Test Data: 7.203019026591527e-05
--------------------------------

--------------------------------
Iteration 3
value: 0.021684994980971976
Rmse of Train Data: 6.080023088490576e-05
Rmse of Test Data: 7.275554821771114e-05
--------------------------------

--------------------------------
Iteration 4
value: 0.021511191082656336
Rmse of Train Data: 6.0026992489393246e-05
Rmse of Test Data: 7.150426560524553e-05
--------------------------------

--------------------------------
Iteration 5
value: 0.021603137364644677
Rmse of Train Data: 6.0552142949353315e-05
Rmse of Test Data: 7.175639141935256e-05
------------------

## none_Parallel_ADMM 2

In [107]:
none_Parallel_ADMM(features,b2,features_test , b2_test,d,100,0.01)

First value: 1.5407880458788372
--------------------------------
--------------------------------
Iteration 1
value: 0.022766153388275603
Rmse of Train Data: 6.769397144484318e-05
Rmse of Test Data: 8.962260179753699e-05
--------------------------------

--------------------------------
Iteration 2
value: 0.021120972265173364
Rmse of Train Data: 6.053504630521694e-05
Rmse of Test Data: 7.790750626202643e-05
--------------------------------

--------------------------------
Iteration 3
value: 0.021698557954550918
Rmse of Train Data: 6.118240692953422e-05
Rmse of Test Data: 7.874698709469054e-05
--------------------------------

--------------------------------
Iteration 4
value: 0.021480872667673432
Rmse of Train Data: 6.036689280058268e-05
Rmse of Test Data: 7.708818344861043e-05
--------------------------------

--------------------------------
Iteration 5
value: 0.021594275877021542
Rmse of Train Data: 6.099384400907771e-05
Rmse of Test Data: 7.799636648665663e-05
-------------------

## Parallel_ADMM

In [112]:
Parallel_ADMM(features,b1,features_test , b1_test,d,100,0.01)

First value: 1353.9604692378248
--------------------------------
--------------------------------
Iteration 1
value: 0.1434455922661623
Rmse of Train Data: 0.000466489077519509
Rmse of Test Data: 0.0005163459184554645
--------------------------------
--------------------------------
Iteration 2
value: 0.09044786296989878
Rmse of Train Data: 0.0002934775547812894
Rmse of Test Data: 0.0003356096238657343
--------------------------------
--------------------------------
Iteration 3
value: 0.18786931647342914
Rmse of Train Data: 0.0006097110702429547
Rmse of Test Data: 0.0006639975680648149
--------------------------------
--------------------------------
Iteration 4
value: 0.12778865808912224
Rmse of Train Data: 0.00041352023761104034
Rmse of Test Data: 0.0004607996042989817
--------------------------------
--------------------------------
Iteration 5
value: 0.1568633187564628
Rmse of Train Data: 0.0005086150166812757
Rmse of Test Data: 0.0005599946224853212
------------------------------

In [109]:
Parallel_ADMM(features,b2,features_test , b2_test,d,100,0.01)

First value: 272.26288112245317
--------------------------------
--------------------------------
Iteration 1
value: 0.15439631174704824
Rmse of Train Data: 0.0005020825324177602
Rmse of Test Data: 0.0005420537733587089
--------------------------------
--------------------------------
Iteration 2
value: 0.09016852277538039
Rmse of Train Data: 0.0002924526369041977
Rmse of Test Data: 0.00032967720038462774
--------------------------------
--------------------------------
Iteration 3
value: 0.18848834331290368
Rmse of Train Data: 0.0006116371003840453
Rmse of Test Data: 0.0006528248815714964
--------------------------------
--------------------------------
Iteration 4
value: 0.12500316798623476
Rmse of Train Data: 0.0004043031313195559
Rmse of Test Data: 0.0004428015866090394
--------------------------------
--------------------------------
Iteration 5
value: 0.15385096741840434
Rmse of Train Data: 0.0004987027379570686
Rmse of Test Data: 0.0005388451483303384
---------------------------

Exception ignored in: <function Pool.__del__ at 0x000002167F510700>
Traceback (most recent call last):
  File "c:\Python39\lib\multiprocessing\pool.py", line 264, in __del__
    if self._state == RUN:
AttributeError: 'ThreadPool' object has no attribute '_state'


--------------------------------
Iteration 39
value: 0.1435945182346531
Rmse of Train Data: 0.0004652576917529983
Rmse of Test Data: 0.0005050519588466412
--------------------------------
--------------------------------
Iteration 40
value: 0.11007139854562963
Rmse of Train Data: 0.00035556039953314365
Rmse of Test Data: 0.0003934345018642023
--------------------------------
--------------------------------
Iteration 41
value: 0.14359451823420857
Rmse of Train Data: 0.0004652576917515487
Rmse of Test Data: 0.0005050519588451761
--------------------------------
--------------------------------
Iteration 42
value: 0.11007139854547679
Rmse of Train Data: 0.0003555603995326447
Rmse of Test Data: 0.00039343450186369634
--------------------------------
--------------------------------
Iteration 43
value: 0.14359451823409658
Rmse of Train Data: 0.0004652576917511835
Rmse of Test Data: 0.000505051958844807
--------------------------------
--------------------------------
Iteration 44
value: 0.

In [110]:
LassoCV_Regresion(features,b1,features_test , b1_test)

--------LassoCV Results--------
Rmse of Train Data: 3.030480529021986e-05
Rmse of Test Data: = 3.736163081038609e-05
--------------------------------


c:\Python39\lib\site-packages\sklearn\utils\validation.py:63: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  return f(*args, **kwargs)


In [111]:
LassoCV_Regresion(features,b2,features_test , b2_test)

--------LassoCV Results--------
Rmse of Train Data: 2.7081595690117215e-05
Rmse of Test Data: = 3.523463290344801e-05
--------------------------------


c:\Python39\lib\site-packages\sklearn\utils\validation.py:63: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  return f(*args, **kwargs)
